In [1]:
from QMLE_scipy import *
from explore import *

### 1. Test k=2

In [2]:
# set up initial guess
sigma = 1
lam = 0.5
gamma = 0.1
rho = 0.2
beta = np.array([1,2])
params = [sigma,lam,gamma, rho, *beta]
params

# set up input data
n = 4
T = 4
k = 2
alpha = np.random.normal(0,1,4)
c0 = np.random.normal(0,1,4)
x1 = []
for i in range(T):
    x1.append(np.random.normal(0,1,n*k).reshape(n,k))
y1 = []
for i in range(T+1):
    y1.append(np.random.normal(0,1,4))
# manually setting up a row-normalized spatial weight vector with 0 diagonals for even time points
w0 = np.array([[0,1,0,0],[1,0,0,0],[0,0,0,1],[0,1,0,0]]).reshape(4,4)
w1 = np.array([[0,0,1,0],[1,0,0,0],[0,1,0,0],[1,0,0,0]]).reshape(4,4)
W_ls = []
for i in range(T+1):
    if is_even(i):
        W_ls.append(w0)
    else:
        W_ls.append(w1)

In [4]:
# run optimization with constraints
result = QMLE_scipy_estimate(x1, y1, W_ls,initial_guess = params,est_coeff = False)

In [5]:
result.parameters

[0.6739760370692636,
 -0.4737341285408044,
 -0.42025286950435436,
 -0.5876667412448603,
 array([ 0.16144904, -0.09739646])]

In [6]:
result.gamma()

-0.42025286950435436

In [7]:
result.beta()

[array([ 0.16144904, -0.09739646])]

In [8]:
result.residual_mean()

0.14551029633298995

In [9]:
result.residual_std()

1.6250143704907318

In [10]:
result.print_table()

Residuals:
     Min        1Q      Median       3Q      Max
--------  --------  ----------  -------  -------
-2.02688  -1.35917  -0.0525139  1.77555  3.05994

Estimated Parameters:
   Sigma     Lambda      Gamma        Rho  Beta
--------  ---------  ---------  ---------  -----------------------------------
0.673976  -0.473734  -0.420253  -0.587667  [array([ 0.16144904, -0.09739646])]

Estimated alpha: 
  alpha_0    alpha_1    alpha_2    alpha_3
---------  ---------  ---------  ---------
-0.316751    1.31824   -1.25436   0.252866

Estimated c0 (display first 10 rows): 
    c0_0      c0_1      c0_2      c0_3
--------  --------  --------  --------
0.159468  0.314142  0.457786  0.255285

Estimated asympotitic variance: 
                          Gamma         Rho      Beta_0      Lambda     Sigma^2
-------  ----------  ----------  ----------  ----------  ----------  ----------
Gamma     0.0839639   0.0525068   0.0326105   0.0469255  -0.119602   -0.0523807
Rho       0.0525068   0.111868    

In [12]:
result = QMLE_scipy_estimate(x1, y1, W_ls,initial_guess = params, constrain= False)

In [13]:
result.print_table()

Residuals:
    Min        1Q      Median       3Q      Max
-------  --------  ----------  -------  -------
-2.0269  -1.35917  -0.0525114  1.77555  3.05995

Estimated Parameters:
   Sigma     Lambda      Gamma        Rho  Beta
--------  ---------  ---------  ---------  -----------------------------------
0.673976  -0.473736  -0.420246  -0.587663  [array([ 0.16145066, -0.09739958])]

Estimated alpha: 
  alpha_0    alpha_1    alpha_2    alpha_3
---------  ---------  ---------  ---------
-0.316759    1.31824   -1.25435   0.252869

Estimated c0 (display first 10 rows): 
    c0_0      c0_1      c0_2      c0_3
--------  --------  --------  --------
0.159465  0.314135  0.457781  0.255281

Estimated asympotitic variance: 
                          Gamma         Rho      Beta_0      Lambda     Sigma^2
-------  ----------  ----------  ----------  ----------  ----------  ----------
Gamma     0.0839637   0.0525074   0.0326102   0.046925   -0.119601   -0.0523804
Rho       0.0525074   0.111869    0.0

### 2. Test k=1

In [14]:
# set up initial guess
initial_guess = [1.5,0.2,0.5,0.3,1.2] # in the order sigma, lam, gamma, rho, beta (all real numbers)

n = 20
k = 1
T = 10

# generate samples
t_theta = [0.1,0.2,np.array([1]).reshape(k,1),0.5,1] # true parameters
t_gamma, t_rho, t_beta, t_lam, t_sig = t_theta
alpha = np.random.normal(0,1,T).reshape(T,1)
c0 = np.random.normal(0,1,n).reshape(n,1)
l_n = np.ones(n).reshape(n,1)
x = []
for i in range(T):
    x.append(np.random.normal(0,1,n).reshape(n,k))
# manually setting up a row-normalized spatial weight vector with 0 diagonals for even time points
w0 = generate_weight_matrix(n)
w1 = generate_weight_matrix(n)

W_ls = []
for i in range(T+1):
    if is_even(i):
        W_ls.append(w0)
    else:
        W_ls.append(w1)

        V_nt = []
for i in range(T):
    V_nt.append(np.random.normal(0,t_sig,n).reshape(n,1))

Y0 = np.random.normal(0,1,n).reshape(n,1)

y = []
y.append(Y0)
for i in range(T):
    c_vec = t_gamma*y[i]+t_rho*np.matmul(W_ls[i],np.array(y[i]).reshape(n,1))+np.matmul(x[i],np.array(t_beta).reshape(k,1))+c0+alpha[i][0]*l_n+V_nt[i]
    Y_nt = np.matmul(np.linalg.inv(np.identity(n)-t_lam*W_ls[i+1]),c_vec).reshape(n,1)
    y.append(Y_nt)

In [15]:
result = QMLE_scipy_estimate(x, y, W_ls,initial_guess = initial_guess, constrain= False)

In [16]:
result.print_table()

Residuals:
     Min       1Q     Median        3Q      Max
--------  -------  ---------  --------  -------
-4.09632  -1.3641  -0.360619  0.705146  3.11206

Estimated Parameters:
   Sigma    Lambda      Gamma       Rho  Beta
--------  --------  ---------  --------  ---------------------
0.924593  0.452931  0.0699691  0.193895  [array([0.84097786])]

Estimated alpha: 
  alpha_0    alpha_1    alpha_2    alpha_3    alpha_4    alpha_5    alpha_6    alpha_7    alpha_8    alpha_9
---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------
 0.165048  -0.586928   0.702952   0.664429   0.191917  -0.691155   -1.27248    0.59183  -0.162627   0.397013

Estimated c0 (display first 10 rows): 
    c0_0       c0_1      c0_2      c0_3       c0_4      c0_5    c0_6     c0_7       c0_8     c0_9
--------  ---------  --------  --------  ---------  --------  ------  -------  ---------  -------
0.200112  -0.630222  0.138985  0.181304  -0.715681  0.434026  0.1779

In [17]:
result = QMLE_scipy_estimate(x, y, W_ls,initial_guess = initial_guess, constrain= False)

In [ ]:
result.print_table